In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!git clone https://github.com/SEERNET/EmoInt.git
%cd EmoInt
%cp -r /content/drive/My\ Drive/MCA\ Project/EmoInt/tweetokenize tweetokenize
!python setup.py install

/content/drive/My Drive/MCA Project/EmoInt
cp: cannot copy a directory, '/content/drive/My Drive/MCA Project/EmoInt/tweetokenize', into itself, 'tweetokenize/tweetokenize'
running install
running bdist_egg
running egg_info
writing EmoInt.egg-info/PKG-INFO
writing dependency_links to EmoInt.egg-info/dependency_links.txt
writing requirements to EmoInt.egg-info/requires.txt
writing top-level names to EmoInt.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'

writing manifest file 'EmoInt.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
copying emoint/featurizers/emoint_featurizer.py -> build/lib/emoint/featurizers
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/emoint
copying build/lib/emoint/__init__.py -> build/bdist.linux-x86_64/egg/emoint
creating build/bdist.linux-x86_64/egg/emoint/featurizers
copying build/lib/emoint/featurizers/afinn_valence_featurizer.py -> build/bdist.linux-x

In [0]:
from EmoInt.emoint.featurizers.emoint_featurizer import EmoIntFeaturizer
from EmoInt.tweetokenize.tweetokenize.tokenizer import Tokenizer
%cd /content/drive/My Drive/MCA Project/

/content/drive/My Drive/MCA Project


In [0]:
!pip install --upgrade gensim
!pip install libsvm

# # import gensim
# # gensim.__version__

In [0]:
import pandas as pd
from gensim.models import FastText
import re
import numpy as np
from libsvm import svmutil
import libsvm
from sklearn.metrics import confusion_matrix, roc_curve, auc
from scipy.stats import kendalltau

In [0]:
model_summary = FastText.load('project/models-4.1/model_summary.bin')

In [0]:
# data = pd.read_excel("Dataset/NewsHeadlines_01012019_30092019.xlsx")
# data.columns =[column.replace(" ", "_") for column in data.columns] 

# ls = [1 if i < 4761 else 0 for i in range(47611)]
# data.sort_values("Like_Count", inplace=True,ascending=False)
# data['class'] = ls
# data["Category"].replace({"Tech": "Technology","-":"Uncategorized", "Sports":"Sport"}, inplace=True)
# data.query(' Post_Type == "photo" or Post_Type == "video" or Post_Type == "link" ', inplace=True)
# data.reset_index(inplace=True,drop=True)
# data = data.truncate(before=0,after=4761*2)
# data = data.sample(frac=1)

In [0]:
# data_ner = pd.read_csv("project/data_ner.csv", sep=";")
# data_pos = pd.read_csv("project/data_pos.csv")
# len(data_pos), len(data_ner)

(9523, 9523)

In [0]:
# ner_tokens = data_ner["ner_tokens"].astype(str).tolist()
arrays = np.load("Final/part_d_features.npy")

In [0]:
# check = (data_ner[data_ner['Post Type'] == 'link'])
# print(check)

In [0]:
# a = data_ner[data_ner['Headline'] == 'Australian motivational speaker Nick Vujicic seen fishing in Pasir Ris']
# b = a['ner_tokens'].tolist()
# print(b[0])

In [0]:
# # #######################################################

# #headline = data["Headline"].tolist()
# headline = data_pos['Headline'].tolist()

# comb_feature = []

# for i in headline:
  
#   #pos_vec = head_vec[headline.index(i)]
#   #print(i)
#   if (data_ner[data_ner['Headline'] == i].empty == False):
#     head = i
#     ner_i = data_ner[data_ner['Headline'] == i]
#     pos_vec = arrays[headline.index(i)]
#     ner_token = ner_i['ner_tokens'].tolist()[0]
#     cat = ner_i['Category'].tolist()[0]
#     pos_type = ner_i['Post Type'].tolist()[0]
#     class_ = ner_i["class"].tolist()[0]

#     comb_feature.append([head,cat,pos_type,ner_token,pos_vec,class_])
    

# df = pd.DataFrame(comb_feature,columns = ['Headline','Category','Post_Type','Ner_Token','POS_Vec', 'class'])
# df.to_csv('Combined_Features.csv')    


In [0]:
combine_f = pd.read_csv('Combined_Features.csv')
len(combine_f)

8966

In [0]:
# !unzip EmoInt.zip

In [0]:
#### SVM for these emotions
import re
category = set(combine_f["Category"].tolist())
post_types = set(combine_f["Post_Type"].tolist())
ner = combine_f["Ner_Token"].astype(str).tolist()
# ner = [re.sub("\"'\]\[(),","", n) for n in ner] 
# print(ner)
headlines= []
for sent_str in combine_f["Headline"]:
#     sent_str = re.sub(r"'", "", str(sent_str).lower())
    tokens = re.sub(r"[^a-z0-9]+", " ", str(sent_str).lower()).split()
    if(tokens == []):
        tokens = "nan"
    headlines.append(tokens)
    
cat_vec = {}
for c in category:
    cat_vec[c] = model_summary.wv[c] 

for c in post_types:
    cat_vec[c] = model_summary.wv[c]
    
head_vec = []
n = len(headlines)
for h in range(n):
    sum1 = np.array([0.0 for i in range(100)])
    for tok in ner[h]:
        sum1+=model_summary.wv[tok[0]]
#     print(sum1)
    sum1 = sum1/len(headlines[h])
#     print(sum1)
    sum1 = np.concatenate((sum1,cat_vec[combine_f["Category"][h]],cat_vec[combine_f["Post_Type"][h]], arrays[h]))
    head_vec.append(sum1)

In [0]:
head_vec = np.array(head_vec)


In [0]:
feat_len = int(0.8*len(head_vec))
train = head_vec[:feat_len]
res_train = combine_f["class"][:feat_len].tolist()
test = head_vec[feat_len:]
res_test = combine_f["class"][feat_len:].tolist()
print(len(test), len(res_test))

1794 1794


In [0]:
model_svm = svmutil.svm_train(res_train, train,'-t 2 -c 2')

In [0]:
y = svmutil.svm_predict(res_test, test, model_svm)

Accuracy = 54.1249% (971/1794) (classification)


In [0]:
confusion_matrix(res_test, y[0], labels=[0,1])

array([[802, 121],
       [702, 169]])

In [0]:
tn, fp, fn, tp = confusion_matrix(res_test, y[0], labels=[0,1]).ravel()

In [0]:
recall = tp/(tp+fn)
recall

0.19402985074626866

In [0]:
precision = tp/(tp+fp)
precision

0.5827586206896552

In [0]:
f1 = precision*recall*2/(precision+recall)
f1

0.2911283376399656

In [0]:
fpr, tpr, thresholds = roc_curve(res_test, y[0])
AUC = auc(fpr, tpr)
AUC

0.5314677964457237

In [0]:
svmutil.svm_save_model("model_all4_classification.bin",model_svm)


In [0]:
svmutil.svm_train(res_train+res_test, list(train)+list(test),'-v 10 -t 2 -c 2')

Cross Validation Accuracy = 54.2382%


54.23823332589783

In [0]:
#With EmoInt-------------------------------------------------------------------------------------------------------------
featurizer = EmoIntFeaturizer()
tokenizer = Tokenizer()

In [0]:
def get_vector(h):
    
    headlines = []
    i= 0
    for sent_str in h:

        print(i)

        i+=1

        features = featurizer.featurize(str(sent_str), tokenizer)
        
        headlines.append(features)
    
    return headlines
head_vec2 = get_vector(headlines)

Streaming output truncated to the last 5000 lines.
3967
3968
3969
3970
3971
3972
3973
3974
3975
3976
3977
3978
3979
3980
3981
3982
3983
3984
3985
3986
3987
3988
3989
3990
3991
3992
3993
3994
3995
3996
3997
3998
3999
4000
4001
4002
4003
4004
4005
4006
4007
4008
4009
4010
4011
4012
4013
4014
4015
4016
4017
4018
4019
4020
4021
4022
4023
4024
4025
4026
4027
4028
4029
4030
4031
4032
4033
4034
4035
4036
4037
4038
4039
4040
4041
4042
4043
4044
4045
4046
4047
4048
4049
4050
4051
4052
4053
4054
4055
4056
4057
4058
4059
4060
4061
4062
4063
4064
4065
4066
4067
4068
4069
4070
4071
4072
4073
4074
4075
4076
4077
4078
4079
4080
4081
4082
4083
4084
4085
4086
4087
4088
4089
4090
4091
4092
4093
4094
4095
4096
4097
4098
4099
4100
4101
4102
4103
4104
4105
4106
4107
4108
4109
4110
4111
4112
4113
4114
4115
4116
4117
4118
4119
4120
4121
4122
4123
4124
4125
4126
4127
4128
4129
4130
4131
4132
4133
4134
4135
4136
4137
4138
4139
4140
4141
4142
4143
4144
4145
4146
4147
4148
4149
4150
4151
4152
4153
4154
4155
4156

In [0]:
hv = []

for i, h in enumerate(list(head_vec)):
  # print(head_vec2[i])
  hv.append(list(h)+ head_vec2[i])
hv = np.array(hv)
hv.shape

(8966, 439)

In [0]:
feat_len = int(0.8*len(head_vec))
train = hv[:feat_len]
res_train = combine_f["class"][:feat_len].tolist()
test = hv[feat_len:]
res_test = combine_f["class"][feat_len:].tolist()
print(len(test), len(res_test))

1794 1794


In [0]:
model_svm = svmutil.svm_train(res_train, train,'-t 2 -c 2')

In [0]:
y = svmutil.svm_predict(res_test, test, model_svm)

Accuracy = 54.5151% (978/1794) (classification)


In [0]:
confusion_matrix(res_test, y[0], labels=[0,1])

array([[566, 357],
       [459, 412]])

In [0]:
tn, fp, fn, tp = confusion_matrix(res_test, y[0], labels=[0,1]).ravel()

In [0]:
recall = tp/(tp+fn)
recall

0.4730195177956372

In [0]:
precision = tp/(tp+fp)
precision

0.5357607282184655

In [0]:
f1 = precision*recall*2/(precision+recall)
f1

0.5024390243902439

In [0]:
fpr, tpr, thresholds = roc_curve(res_test, y[0])
AUC = auc(fpr, tpr)
AUC

0.5431186429714915

In [0]:
svmutil.svm_save_model("model_all5_classification.bin",model_svm)

In [0]:
svmutil.svm_train(res_train+res_test, list(train)+list(test),'-v 10 -t 2 -c 2')